In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget
%pdb off

from pyCascade import probePost, physics, utils, probeReadWrite
from pyCascade.probeReadWrite import read_probes_file_switch
from filloutVentilationStats import *
from matplotlib import pyplot as plt
from matplotlib import cm, colors
import matplotlib.ticker as ticker
import numpy as np
import scipy as sp
import os
from IPython.core.debugger import set_trace
import pandas as pd
import seaborn as sns
from cycler import cycler
import plotly.express as px
import plotly
import plotly.graph_objects as go
from plotly.offline import plot
from plotly.subplots import make_subplots
from IPython.display import display, HTML
import statsmodels.api as sm
import warnings

plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

plt.rcParams['figure.dpi'] = 140
im_scaling = .75
plt.rcParams['figure.figsize'] = [6.4 * im_scaling, 4.8 * im_scaling]

############ Universal ################
scratch_home = os.getenv('SCRATCH') #need to set SCRATCH (even if there is no real SCRATCH) to the location where results are written
scratch_dir = f'{scratch_home}/Cascade/city_block_cfd'
home_dir = !pwd
home_dir = home_dir[0]

display(scratch_dir)
display(home_dir)
plt.close('all')

## Runs

In [2]:


multiRun_dir = f"{home_dir}/CHARLES/multiRuns/"
plotFolder = f"{multiRun_dir}"

## Read in results

In [3]:
overWrite = False
if os.path.exists(f"{multiRun_dir}/roomVentilationMI.csv") == False or os.path.exists(f"{multiRun_dir}/flowStatsMI.csv") == False or overWrite:

    runs = {
        253: {'A': 45, 'WS': 2, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        254: {'A': 45, 'WS': 4, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        255: {'A': 0,  'WS': 2, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        256: {'A': 0,  'WS': 4, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        321: {'A': 45, 'WS': 2, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        322: {'A': 45, 'WS': 4, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        323: {'A': 0,  'WS': 2, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        324: {'A': 0,  'WS': 4, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
    }

    flowStatsMI, roomVentilationMI = readRunStats(runs, home_dir, scratch_dir, multiRun_dir = plotFolder)
else:
    flowStatsMI = pd.read_csv(f"{multiRun_dir}/flowStatsMI.csv", index_col = [0,1])
    roomVentilationMI = pd.read_csv(f"{multiRun_dir}/roomVentilationMI.csv", index_col = [0,1])

In [ ]:
foo = roomVentilationMI["q-D-room"] / roomVentilationMI["q-D-room-Norm"]
# histogram
fig, ax = plt.subplots()
ax.hist(foo, bins=np.arange(0.001, 21.001), color='c', edgecolor='black', linewidth=1.2)

# Violins


## Wall Convection Violins

In [ ]:
value_vars = ["mean-mass_flux(S)-roomCeil-perA-Norm", "mean-mass_flux(S)-roomWalls-perA-Norm", "mean-mass_flux(S)-roomFloor-perA-Norm"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)
plotdf["value"]

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.5, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()


value_vars = ["mean-mass_flux(S)-roomCeil-h-Norm", "mean-mass_flux(S)-roomWalls-h-Norm", "mean-mass_flux(S)-roomFloor-h-Norm"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)
plotdf["value"]

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.5, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()


## Volume Violins

In [ ]:
rooms = ["room5", "room4", "room3", "room2", "room1", "room0"]
value_vars = [f"mean-S-{r}" for r in rooms]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)
# plotdf["value"]

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

value_vars = [f"mean-S-room"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)
# plotdf["value"]

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()


### Temperature-Tracer Comparison/Check

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == False]

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.scatterplot, "c-T-room", "mean-S-room", "delT", hue_order = [0, 5])
g.add_legend()


## Tracer Decay Violins

In [ ]:

value_vars = [f"tau-D-{r}" for r in rooms]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

value_vars = [f"tau-D-room"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

value_vars = [f"tau-D-room"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["delT"] == 5]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "SS", order = value_vars, hue_order = [True, False], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

In [ ]:

value_vars = [f"q-D-{r}-Norm" for r in rooms]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

value_vars = [f"q-D-room-Norm"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

value_vars = [f"q-D-room-Norm"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["delT"] == 5]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "SS", order = value_vars, hue_order = [True, False], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"].copy()
plotdf = plotdf[plotdf["SS"] == True]
plotdf["mean-mass_flux(S)-Norm"] /= -rho
plotdf["mean-mass_flux-Norm"] /= rho
plotdf["q-D-room-Norm"] /= rho

qois = ["mean-mass_flux-Norm", "mean-sn_prod(abs(u))-Norm", "mean-mass_flux(S)-Norm", "q-D-room-Norm"]

group = ["roomType", "WS", "Ri", "delT", "houseType", "C", "AofA"]

plotdfGrouped = plotdf.groupby(group, as_index=False).mean(numeric_only=True)

x_var = "roomType"
plt.figure(figsize=(10, 6))

# Create a new column to identify each QOI
plotdfMelted = plotdfGrouped.melt(id_vars=[x_var], value_vars=qois, var_name="QOI", value_name="Value")

# Create the box plot with QOI as hue
sns.boxplot(data=plotdfMelted, x=x_var, y="Value", hue="QOI", palette="Set2")

# Customize the plot
plt.legend(title="QOI", loc='upper right')
# map legend to new labels
new_labels = ["$\\bar{Q}$", "$\overline{|Q|}$", "$\\bar{Q}_{s}$", "$\\bar{Q}_d$"]
for t, l in zip(plt.legend().texts, new_labels): t.set_text(l)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.xlabel("Room Type", fontsize=14)
plt.ylabel("Normalized Ventilation Rate", fontsize=14)
plt.title("Box Plot for Different QOIs", fontsize=16)
plt.tight_layout()

## Decay Heat Flux Violins

In [ ]:

value_vars = [f"q-D-{r}-Norm" for r in rooms]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

value_vars = [f"q-D-room-Norm"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["delT"] == 5]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "SS", order = value_vars, hue_order = [True, False], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

## Mass Flux Violins

In [ ]:
value_vars = [f"mean-mass_flux-Norm"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["delT"] == 5]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "SS", order = value_vars, hue_order = [True, False], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

In [ ]:
plotdf_all = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf_all = plotdf[plotdf["SS"] == True]
plotdf_all["mean-mass_flux(S)-Norm"] /= -rho
plotdf_all["mean-mass_flux-Norm"] /= rho
plotdf_all["q-D-room-Norm"] /= rho

r_lim = [.59, .59, .59, .059]
# r_lim = [r/2 for r in r_lim]
fig, axs = plt.subplots(4, 2, subplot_kw={'projection': 'polar'}, figsize = (10, 16), dpi = 300, layout="tight")
colors = plt.cm.get_cmap("Set2").colors  # Use Matplotlib's Set2 colormap

# Helper function to convert 'rgb(r, g, b)' to (r/255, g/255, b/255) tuple
def rgb_to_mpl(rgb_str):
    rgb = rgb_str.replace('rgb(', '').replace(')', '').split(',')
    return tuple(int(val) / 255 for val in rgb)

for i, room in enumerate(["corner", "cross", "dual", "single"]):
    for j, delT in enumerate([0, 5]):
        plotdf = plotdf_all.copy()
        plotdf = plotdf_all[plotdf_all["roomType"] == room]
        plotdf = plotdf[plotdf["delT"] == delT]
    
        x_var = "AofA"
        y_vars = ["mean-mass_flux-Norm", "mean-sn_prod(abs(u))-Norm", "mean-mass_flux(S)-Norm", "q-D-room-Norm"] #"mean-sn_prod(abs(u))-Norm"

        # Create the polar plot
        ax = axs[i, j]
        ax.set_ylim(0, r_lim[i])
    
        # Customize gridlines to be lighter
        ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
        # Set the radial labels to use scientific notation
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax.set_rlabel_position(0)

        # Create the bars
        plotdfGrouped = plotdf.groupby(x_var, as_index=False).median(numeric_only=True) #CHANGED TO MEDIAN
        angles = np.deg2rad(plotdfGrouped[x_var])
        radii = np.mean(plotdfGrouped[y_vars].values, axis = 1)
        ax.bar(angles, radii, width=np.pi/6, color='white', edgecolor='black', linewidth = 0.25) #CHANGED TO MEDIAN
        for k, y_var in enumerate(y_vars):
            width = np.pi/35
            offset = (len(y_vars)/2.5 - k)*width
            angles = np.deg2rad(plotdfGrouped[x_var]) + offset
            radii = plotdfGrouped[y_var].values
            # color = "lightgrey"
            color = colors[k]
            ax.bar(angles, radii, width=width, color=color, edgecolor = 'white', linewidth = .5, alpha = 1)
        
        # Adding a legend for the custom error bars
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        # ax.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))

        # Customize plot
        ax.set_theta_direction(1)  # counter clockwise
        ax.set_theta_offset(np.pi)  # Starting from the right
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.yaxis.get_offset_text().set_fontsize(16)  # X-axis scientific notation font size
        yticks = ax.get_yticks()
        ax.set_yticks(yticks[1::2])

# plt.tight_layout()

In [ ]:
n_layers = 6

r_lim = [.59, .59, .59, .059]
r_lim = [r/n_layers for r in r_lim]
# r_lim = [r/2 for r in r_lim]
fig, axs = plt.subplots(4, 2, subplot_kw={'projection': 'polar'}, figsize = (10, 16), dpi = 300, layout="tight")
safe_colors = px.colors.qualitative.Safe

# Helper function to convert 'rgb(r, g, b)' to (r/255, g/255, b/255) tuple
def rgb_to_mpl(rgb_str):
    rgb = rgb_str.replace('rgb(', '').replace(')', '').split(',')
    return tuple(int(val) / 255 for val in rgb)

for i, room in enumerate(["corner", "cross", "dual", "single"]):
    for j, delT in enumerate([0, 5]):
        plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
        plotdf = plotdf[plotdf["SS"] == True]
        plotdf = plotdf[plotdf["roomType"] == room]
        plotdf = plotdf[plotdf["delT"] == delT]
    
        x_var = "AofA"
        y_var = "q-D-room-Norm" #"mean-sn_prod(abs(u))-Norm"

        # Create the polar plot
        ax = axs[i, j]
        ax.set_ylim(0, r_lim[i])
    
        # Customize gridlines to be lighter
        ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
        # Set the radial labels to use scientific notation
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax.set_rlabel_position(0)

        # Create the bars
        plotdfGrouped = plotdf.groupby(x_var, as_index=False).median(numeric_only=True) #CHANGED TO MEDIAN
        angles = np.deg2rad(plotdfGrouped[x_var])
        radii = plotdfGrouped[y_var].values / n_layers
        ax.bar(angles, radii, width=np.pi/6, color='white', edgecolor='black', linewidth = 0.25) #CHANGED TO MEDIAN
        for k in range(n_layers):
            width = np.pi/35
            offset = (n_layers/2.5 - k)*width
            angles = np.deg2rad(plotdfGrouped[x_var]) + offset
            radii = plotdfGrouped[f"q-D-room{k}-Norm"].values
            # color = "lightgrey"
            color = rgb_to_mpl(safe_colors[k])
            ax.bar(angles, radii, width=width, color=color, edgecolor = 'white', linewidth = .5, alpha = 1)
        
        # Adding a legend for the custom error bars
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        # ax.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))

        # Customize plot
        ax.set_theta_direction(1)  # counter clockwise
        ax.set_theta_offset(np.pi)  # Starting from the right
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.yaxis.get_offset_text().set_fontsize(16)  # X-axis scientific notation font size
        yticks = ax.get_yticks()
        ax.set_yticks(yticks[1::2])

# plt.tight_layout()